In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Drawing specifications for lighter mesh
dot_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)  # Small dots
line_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # Thin lines

# Initialize Pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to detect leaning forward or backward
def detect_leaning(landmarks):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    shoulder_midpoint = [(left_shoulder.x + right_shoulder.x) / 2, (left_shoulder.y + right_shoulder.y) / 2]
    hip_midpoint = [(left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2]

    horizontal_alignment = hip_midpoint[0] - shoulder_midpoint[0]

    forward_threshold = -0.05
    backward_threshold = 0.05

    if horizontal_alignment > forward_threshold:
        return "Leaning Forward", horizontal_alignment
    elif horizontal_alignment < backward_threshold:
        return "Leaning Backward", horizontal_alignment

    return "Upright", horizontal_alignment

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1.x - point2.x)*2 + (point1.y - point2.y)*2)

# Function to detect if arms are folded
def are_arms_folded(landmarks, threshold=0.3):
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

    left_wrist_right_elbow_dist = calculate_distance(left_wrist, right_elbow)
    right_wrist_left_elbow_dist = calculate_distance(right_wrist, left_elbow)

    if left_wrist_right_elbow_dist < threshold and right_wrist_left_elbow_dist < threshold:
        return True
    return False

# Function to detect if legs are crossed by checking knee-hip proximity
def are_legs_crossed(landmarks, threshold=0.1):
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Calculate the distances between knees and opposite hips
    left_knee_right_hip_dist = calculate_distance(left_knee, right_hip)
    right_knee_left_hip_dist = calculate_distance(right_knee, left_hip)

    # Check if either distance is below the threshold
    if left_knee_right_hip_dist < threshold or right_knee_left_hip_dist < threshold:
        return True
    return False

# Set up webcam
cap = cv2.VideoCapture(0)

# Initialize Pose
with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Process pose landmarks
        pose_results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Process pose landmarks
        if pose_results.pose_landmarks:
            landmarks = pose_results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

            back_angle = calculate_angle(shoulder, hip, knee)
            leaning_status, horizontal_alignment = detect_leaning(landmarks)

            # Display leaning status
            if leaning_status == "Leaning Forward":
                color = (0, 0, 255)
            elif leaning_status == "Leaning Backward":
                color = (255, 0, 0)
            else:
                color = (0, 255, 0)

            cv2.putText(image, f'{leaning_status} HA: {horizontal_alignment:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

            # Display slouching detection
            if 95 <= back_angle <= 110:
                cv2.putText(image, "Upright", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            elif back_angle < 95:
                cv2.putText(image, "Slouching Detected!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Detect and display if arms are folded
            arms_folded = are_arms_folded(landmarks)
            if arms_folded:
                cv2.putText(image, "Arms Folded!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(image, "Arms Not Folded", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Detect and display if legs are crossed
            legs_crossed = are_legs_crossed(landmarks)
            if legs_crossed:
                cv2.putText(image, "Legs Crossed!", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(image, "Legs Not Crossed", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw pose landmarks on the image
            mp_drawing.draw_landmarks(image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the output
        cv2.imshow('Posture and Leg Cross Detection', image)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resourcesq
cap.release()
cv2.destroyAllWindows()


C:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\pc\AppData\Local\Temp\ipykernel_10388\3852647207.py:53: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt((point1.x - point2.x)*2 + (point1.y - point2.y)*2)
